In [1]:
import dynamo as dyn
import scanpy as sc 

import numpy as np
import pandas as pd
import seaborn as sns

from dynamo.simulation.evaluation import evaluate
from dynamo.tools.connectivity import adj_to_knn
from dynamo.tools.utils import get_finite_inds

from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from scipy.sparse import issparse, csr_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


/Users/random/opt/anaconda3/envs/dynamo-dev/lib/python3.9/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.3 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.3, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [2]:
adata_labeling_obs = pd.read_csv('./adata_labeling_all_combined_processed_final.csv', index_col=0)

adata_labeling_obs.head(2)

,batch,time,cell_type,nGenes,nCounts,pMito,pass_basic_filter,new_Size_Factor,initial_new_cell_size,total_Size_Factor,...,pca_ddhodge_potential,umap_ori_ddhodge_div,umap_ori_ddhodge_potential,curl_umap_ori,divergence_umap_ori,control_point_umap_ori,inlier_prob_umap_ori,obs_vf_angle_umap_ori,n_counts,mt_frac
barcode,,,,,,,,,,,,,,,,,,,,,
CCACAAGCGTGC-JL12_0,JL12_0,3,Mon,1196,1944.0,0.114198,True,1.039024,852.0,1.227772,...,0.507833,-16.341991,1.792488,-0.046206,0.112111,False,0.999549,0.176351,3466.0,0.073572
CCATCCTGTGGA-JL12_0,JL12_0,3,Meg,1172,1706.0,0.067995,True,0.506098,415.0,1.002834,...,4.700147,41.514399,3.392540,0.098062,-0.273977,False,0.860345,0.354914,2831.0,0.048746


In [3]:
################################### Our data ###################################
adata_dyn = dyn.read('./hsc_all_combined_all_layers.h5ad')
adata_filtered2 = dyn.read('./hsc_adata_filtered2.h5ad')
adata_dyn.obs['time'] = 3
adata_dyn.obs.loc[adata_dyn.obs.batch == 'JL_10', 'time'] = 5
adata_labeling = adata_dyn[adata_filtered2.obs_names]

adata_labeling.layers['new'] = adata_labeling.layers['labeled_TC'].copy()

# adata_labeling.X = adata_labeling.layers['labeled_TC'].copy()
# adata_labeling.X[adata_labeling.obs.batch == 'JL_10', :] *= 3/5 

layers_keys = adata_labeling.layers.keys()
for i in ['X_l_TC', 'X_n_TC', 'ambiguous', 'labeled_TC', 'sl_TC', 'sn_TC', 'ul_TC', 'un_TC', 'unlabeled_TC']:
    if i in layers_keys: del adata_labeling.layers[i]

import pickle
union_genes = pickle.load(open('./union_genes.p', 'rb'))

/Users/random/opt/anaconda3/envs/dynamo-dev/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [10]:
len(union_genes), union_genes[:10]

(2574,
 ['AL158064.1',
  'CD163L1',
  'SNX25',
  'TBX19',
  'AC009495.1',
  'CD33',
  'LINC01916',
  'CARD9',
  'RAPGEFL1',
  'OSGEPL1-AS1'])

In [5]:
# ZFPM1: FOG1; EKLF: KLF1; cJun: JUN; EgrNab: NAB2
# ETO2: CBFA2T3; TEL: ETV6 
network_inference_genes = ["GATA2", "GFI1", "GFI1B", "NFE2", "TAL1", "GATA1", "LYL1", "ERG", "FLI1", "MEIS1", "SPI1", "MITF", "LMO2", "LDB1", "RUNX1", "ETV6", "HHEX", "CBFA2T3"]
Krumsiek_11 = ["GATA2", "GATA1", "ZFPM1", "SPI1", "FLI1", "KLF1", "TAL1", "CEBPA", "GFI1", "JUN", "NAB2"]

union_genes += network_inference_genes
union_genes += Krumsiek_11

In [6]:
len(np.unique(union_genes)), len(union_genes)

(2552, 2574)

In [24]:
set(adata_labeling_obs.index).difference(adata_labeling.obs_names)

set()

In [45]:
adata_labeling_obs.index = np.array(adata_labeling_obs.index, dtype=str)

In [46]:
obs_names = adata_labeling.obs_names.to_list()

In [47]:
len(obs_names), len(adata_labeling_obs.index)

(1979, 1947)

In [49]:
# set(adata_labeling_obs.index).difference(obs_names)
set(obs_names).difference(adata_labeling_obs)

1979

In [33]:
adata_labeling_obs.loc[obs_names]

KeyError: "['GCCAAAATCTTA-JL12_0', 'GGCCATTCAATG-JL12_0', 'GGCCTTTATTTC-JL12_0', 'GGCGCCAATCCA-JL12_0', 'GGCGGCGCCAGA-JL12_0', 'GGCGGGTCGGGA-JL12_0', 'GGGCAAGCACAC-JL12_0', 'GGGCACCGGACT-JL12_0', 'GGGCATGTTTTT-JL12_0', 'GGGCCGTAAGAG-JL12_0', 'GGGCTAGAGAGG-JL12_0', 'GGGGCGACTCAT-JL12_0', 'GGGGCGTAAGAG-JL12_0', 'GGGGGGGTTCCC-JL12_0', 'GGGGGTGCGCCG-JL12_0', 'GGGTACTTCGGG-JL12_0', 'GGGTGCTCTACA-JL12_0', 'GGGTGGGTTCCC-JL12_0', 'GGGTGGTCCCGG-JL12_0', 'GGTATCGCCAAC-JL12_0', 'GGTCTAGCCGGA-JL12_0', 'GGTGACGACAAA-JL12_0', 'GGTGATTTGGTT-JL12_0', 'GGTGCCGACTGG-JL12_0', 'GGTGTAATCTCT-JL12_0', 'GGTTCGGATTCA-JL12_0', 'GGAGAAGCACAC-JL_10', 'GGAGCTGCTTCG-JL_10', 'GGCATATCATAG-JL_10', 'GGCGACGTCCTC-JL_10', 'GGCGAGTGTTGC-JL_10', 'GGGTTTGGTTCG-JL_10'] not in index"

In [11]:
adata_labeling.obs['cell_type'] = list(adata_labeling_obs.loc[adata_labeling.obs_names, "cell_type"])

KeyError: "['GCCAAAATCTTA-JL12_0', 'GGCCATTCAATG-JL12_0', 'GGCCTTTATTTC-JL12_0', 'GGCGCCAATCCA-JL12_0', 'GGCGGCGCCAGA-JL12_0', 'GGCGGGTCGGGA-JL12_0', 'GGGCAAGCACAC-JL12_0', 'GGGCACCGGACT-JL12_0', 'GGGCATGTTTTT-JL12_0', 'GGGCCGTAAGAG-JL12_0', 'GGGCTAGAGAGG-JL12_0', 'GGGGCGACTCAT-JL12_0', 'GGGGCGTAAGAG-JL12_0', 'GGGGGGGTTCCC-JL12_0', 'GGGGGTGCGCCG-JL12_0', 'GGGTACTTCGGG-JL12_0', 'GGGTGCTCTACA-JL12_0', 'GGGTGGGTTCCC-JL12_0', 'GGGTGGTCCCGG-JL12_0', 'GGTATCGCCAAC-JL12_0', 'GGTCTAGCCGGA-JL12_0', 'GGTGACGACAAA-JL12_0', 'GGTGATTTGGTT-JL12_0', 'GGTGCCGACTGG-JL12_0', 'GGTGTAATCTCT-JL12_0', 'GGTTCGGATTCA-JL12_0', 'GGAGAAGCACAC-JL_10', 'GGAGCTGCTTCG-JL_10', 'GGCATATCATAG-JL_10', 'GGCGACGTCCTC-JL_10', 'GGCGAGTGTTGC-JL_10', 'GGGTTTGGTTCG-JL_10'] not in index"

In [ ]:
adata_labeling.obs['cell_type'].unique()

In [ ]:
adata_labeling.obs['cell_type'] = adata_labeling.obs['cell_type'].map({
    "GMP": "GMP-like",
    "MPP": "MEP-like",
    'Mon': "Mon", 
    'Meg': "Meg", 
    'Ery': "Ery", 
    'Bas': "Bas", 
    'HSC': "HSC", 
    'Neu': "Neu"
})

adata_labeling.obs['cell_type'].value_counts()

In [ ]:
adata_labeling

In [ ]:
dyn.pp.recipe_monocle(adata_labeling,
                      tkey='time',
                      experiment_type='one-shot',
                      genes_to_use=union_genes,
                      n_top_genes=len(union_genes), 
                      maintain_n_top_genes=True
                     )
adata_labeling.var.use_for_pca.sum()

In [ ]:
list(set(Krumsiek_11 + network_inference_genes).difference(adata_labeling.var_names[adata_labeling.var.use_for_pca]))

In [ ]:
list(set(Krumsiek_11 + network_inference_genes).intersection(adata_labeling.var_names[adata_labeling.var.use_for_pca]))

In [ ]:
dyn.tl.moments(adata_labeling, group='time')
M_s = adata_labeling.layers['M_s'].copy()
M_u = adata_labeling.layers['M_u'].copy()

# here delete the splicing data
adata_full = adata_labeling.copy()
layers_keys = adata_labeling.layers.keys()
for i in ['spliced', 'unspliced', 'X_spliced', 'X_unspliced',  'M_u', 'M_uu', 'M_s', 'M_us', 'M_t', 'M_tt', 'M_n', 'M_tn', 'M_ss', 'M_nn']:
    if i in layers_keys: del adata_labeling.layers[i]
adata_labeling.uns['pp']
adata_labeling.uns['pp']['has_splicing'] = False

dyn.tl.dynamics(adata_labeling, group='time', one_shot_method='sci_fate', model='deterministic')
dyn.tl.reduceDimension(adata_labeling)
dyn.tl.leiden(adata_labeling)
dyn.tl.cell_velocities(adata_labeling,
                       enforce=True,
                       method='cosine',
                       )

dyn.pl.streamline_plot(
    adata_labeling,
    color=['batch'], #+ list(adata_dyn.var_names.intersection(chen_paper_genes)),
    ncols=4,
    basis='umap',
)

In [ ]:
adata_labeling.uns['pp']

In [ ]:
adata_labeling.obs.time.unique()

In [ ]:
#############################################
# alpha = new / (1 - e^{-rt}) * r
#############################################
new = adata_labeling[:, adata_labeling.var.use_for_pca].layers['M_n']
gamma = adata_labeling[:, adata_labeling.var.use_for_pca].var.gamma
time_3_gamma = adata_labeling[:, adata_labeling.var.use_for_pca].var.time_3_gamma.astype(float)
time_5_gamma = adata_labeling[:, adata_labeling.var.use_for_pca].var.time_5_gamma.astype(float)

t = adata_labeling.obs.time.astype(float)
M_s = M_s[:, adata_labeling.var.use_for_pca]

time_3_cells = adata_labeling.obs.time == 3
time_5_cells = adata_labeling.obs.time == 5

def alpha_minus_gamma_s(new, gamma, t, M_s):
    alpha = new.A.T / (1 - np.exp(- gamma.values[:, None] * t.values[None, :])) * gamma.values[:, None]
    gamma_s = gamma.values[:, None] * M_s.A.T
    alpha_minus_gamma_s = alpha - gamma_s

    return alpha_minus_gamma_s

time_3_tmp = alpha_minus_gamma_s(new[time_3_cells, :], time_3_gamma, t[time_3_cells], M_s[time_3_cells, :])
time_5_tmp = alpha_minus_gamma_s(new[time_5_cells, :], time_5_gamma, t[time_5_cells], M_s[time_5_cells, :])

tmp = adata_labeling.layers['velocity_N'].copy()

valid_tmp = tmp[:, adata_labeling.var.use_for_pca].copy()
valid_tmp[time_3_cells, :] = time_3_tmp.T
valid_tmp[time_5_cells, :] = time_5_tmp.T
tmp[:, adata_labeling.var.use_for_pca] = valid_tmp.copy()

adata_labeling.layers['velocity_alpha_minus_gamma_s'] = tmp.copy()

dyn.tl.cell_velocities(adata_labeling,
                       enforce=True,
                       X=adata_labeling.layers['M_t'],
                       V=adata_labeling.layers['velocity_alpha_minus_gamma_s'],
                       method='cosine',
                       )

chen_paper_genes = ['PLEK', 'HBB', 'MPO', 'SPIB', 'CD79A', 'DNTT', 'CD34', 'CD164', "LMO4", "S100A8", "TGFBI"]
dyn.pl.streamline_plot(
    adata_labeling,
    color=['batch'] + list(adata_dyn.var_names.intersection(chen_paper_genes)),
    ncols=4,
    basis='umap',
)

In [ ]:
adata_labeling.obsm['X_umap_ori'] = adata_filtered2.obsm['X_umap'].copy()

dyn.tl.cell_velocities(adata_labeling,
                       # X=adata_labeling.layers['M_t'],
                       # V=adata_labeling.layers['alpha_minus_gamma_s'],
                       basis='umap_ori',
                       method='cosine',
                       # enforce=True,
                       )

dyn.pl.streamline_plot(
    adata_labeling,
    color=['batch'], #+ list(adata_dyn.var_names.intersection(chen_paper_genes)),
    ncols=4,
    basis='umap_ori',
)

In [ ]:
# adata_labeling.obsm['X_pca'] = adata_filtered2.obsm['X_pca'].copy()
# dyn.tl.leiden(adata_labeling, obsm_key='X_pca')
dynamo_revision_genes = ["LYZ", "PLEK", "MPO", "LMO4", "HBB", "cell_type"]

dyn.tl.leiden(adata_labeling, obsm_key='X_umap_ori', result_key='umap_leiden')
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori',
                       color=['umap_leiden', 'cell_type'] + list(adata_dyn.var_names.intersection(dynamo_revision_genes)))

In [ ]:
dyn.tl.louvain(adata_labeling, obsm_key='X_umap_ori', result_key='umap_louvain', resolution=0.5)
dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori',
                       color=['umap_leiden', 'batch', 'umap_louvain'] + list(adata_dyn.var_names.intersection(dynamo_revision_genes)))

In [ ]:
dyn.tl.louvain(adata_labeling, obsm_key='X_umap_ori', result_key='umap_louvain', resolution=0.5)
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', color=['batch', 'umap_louvain'])

In [ ]:
sum(adata_labeling.obs.umap_louvain == '12')

In [ ]:
# adata_labeling.obs['cell_type'] = adata_labeling.obs['umap_louvain'].map({
#     "4": "HSC",
#     "5": "HSC",
#     "12": "HSC",
#     "13": "HSC",
#     "16": "HSC",
#     "10": "HSC",
#     "12": "HSC",
    
    
#     "10": "GMP",
#     "16": "GMP",
    
    
#     "3": 'MPP',
#     "9": "MPP",
#     "14": "MPP",
#     "17": "MPP",
    
#     "19": "Neu",
    
#     "1": "Mon",
#     "7": "Mon",
#     "8": "Mon",
#     "18": "Mon",

#     "6": "Bas",
#     "15": "Bas",
    
#     "2": "Ery",
#     "11": "Ery",

#     "0": "Meg",
# })

dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori',
                       color=['cell_type', 'umap_louvain'])


# save streamline layers

In [ ]:

dyn.configuration.set_pub_style(scaler=2.5)
dyn.pl.streamline_plot(adata_labeling, 
                       ncols=6,
                       figsize=(5, 4),
                       color=dynamo_revision_genes,
                       basis='umap_ori',
                       save_show_or_return='save',
                       save_kwargs={"prefix": './figure/all_data_labeling_streamline'}
                      )

In [ ]:
adata_labeling

# show the phase plot of the new and total RNA 

In [ ]:
# dyn.convert2float(adata_labeling, ['time_3_gamma', 'time_3_gamma_b'], var=True)

In [ ]:
adata_labeling.layers['velocity_U'] = adata_labeling.layers['velocity_alpha_minus_gamma_s'].copy()

In [ ]:
adata_labeling.var['gamma_k'] = adata_labeling.var['time_3_gamma'].astype(float).copy()
adata_labeling.var['gamma_b'] = adata_labeling.var['time_3_gamma_b'].copy()

dyn.pl.phase_portraits(adata_labeling,
                       basis='umap_ori',
                       figsize=(5, 4),
                       genes=dynamo_revision_genes + ['PF4', 'LYZ'],
                       color='cell_type',
                       show_quiver=True, 
                       no_vel_u=True,
                       vkey='U',
                       ekey='M_t',
                       save_show_or_return='save',
                       save_kwargs={"prefix": './figure/all_data_labeling_phase_portrait'}
                      )

In [ ]:
dyn.pl.phase_portraits(adata_labeling,
                       basis='umap_ori',
                       figsize=(5, 4),
                       genes=['PF4', 'LYZ'],
                       color='cell_type',
                       show_quiver=True, 
                       no_vel_u=True,
                       vkey='U',
                       ekey='M_t',
                       save_show_or_return='save',
                       save_kwargs={"prefix": './figure/all_data_labeling_phase_portrait_two_genes'}
                      )

# save all data for downstream vector field analysis, etc. 

In [ ]:
dyn.cleanup(adata_labeling)
adata_labeling.write_h5ad('./adata_labeling_all_combined_processed_final.h5ad')

# plot the heatmap of known HSC related genes 

In [ ]:
gene2celltype = pickle.load(open('gene2celltype.p', 'rb'))

In [ ]:
adata_labeling.var_names.intersection(gene2celltype.keys())

In [ ]:
dynamo_revision_genes

In [ ]:
sc.pl.heatmap(adata_labeling, 
              adata_labeling.var_names.intersection(gene2celltype.keys()), 
              groupby='cell_type', 
              show_gene_labels=True, 
              layer='M_t',
              swap_axes=True)

In [ ]:
sc.pl.heatmap(adata_labeling, 
              adata_labeling.var_names.intersection(gene2celltype.keys()), 
              groupby='cell_type', 
              show_gene_labels=True, 
              layer='M_n',
              swap_axes=True)

In [ ]:
sc.tl.rank_genes_groups(adata_labeling, 'cell_type', method='t-test', layer='M_n', use_raw=False)
sc.set_figure_params(dpi=150)
sc.pl.rank_genes_groups(adata_labeling, n_genes=25, sharey=False)


In [ ]:
example_genes = ["CSF2RB", "HDC", 
                "MINPP1", "XACT", "TFRC",
                "NKTR",
                "HMGB1", "HMGB1P5",
                "MTATP6P1",
                "SOS1", "RRN3", "PRKAR2B",
                "MT-CO3", "GSTP1",
                "AC011825.3"]

dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori', 
                       color=example_genes)

In [ ]:
sc.tl.rank_genes_groups(adata_labeling, 'cell_type', method='t-test', layer='M_t', use_raw=False)
sc.set_figure_params(dpi=150)
sc.pl.rank_genes_groups(adata_labeling, n_genes=25, sharey=False)


In [ ]:
gene_list = sc.get.rank_genes_groups_df(adata_labeling, group="Bas").head(100)['names'].to_list()
valid_gene_list = list(np.array(gene_list)[~ np.array([True if i.startswith("MT") else False for i in gene_list])])
valid_gene_list = list(gene_list)

res = dyn.ext.enrichr(valid_gene_list, organism='human')

res.res2d.head(20)

In [ ]:
example_genes = ["CSF2RB", "HDC", 
                "FBXO7", "EIF5A", "MINPP1",
                "EEF1A1P5", "RPS3", "PABPC1",
                "RPL41",
                "NCL",
                "SOS1", "PDLIM1",
                "SMIM24", "GSTP1",
                "OR5G5P", "INO80D", "AC002101.1"]

dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori', 
                       color=example_genes)

# differential geometry, etc below: 

In [ ]:
adata_labeling, adata_filtered2

In [ ]:
# adata_labeling.obsm['X_pca'] = adata_filtered2.obsm['X_pca'].copy()
# adata_labeling.uns['pca'] = adata_filtered2.uns['pca'].copy()
# adata_labeling.varm['PCs'] = adata_filtered2.varm['PCs'].copy()
dyn.tl.cell_velocities(adata_labeling,
                       X=adata_labeling.layers['M_t'],
                       V=adata_labeling.layers['velocity_alpha_minus_gamma_s'],
                       basis='pca',
                       method='cosine',
                       enforce=True,
                       )

In [ ]:
adata_labeling

In [ ]:
adata_filtered2.varm['PCs'].shape, adata_filtered2.uns['pca'], adata_filtered2.obsm['X_pca'].shape

In [ ]:
dyn.vf.VectorField(adata_labeling, basis='pca', M=1000)
dyn.ext.ddhodge(adata_labeling, basis='pca')

In [ ]:
adata_labeling

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', color='pca_ddhodge_potential')

In [ ]:
dyn.vf.VectorField(adata_labeling, basis='umap_ori', map_topography=True, M=1000, pot_curl_div=True)

In [ ]:
adata_labeling

In [ ]:
dyn.pl.topography(adata_labeling, basis='umap_ori', fps_basis='umap_ori', color=['pca_ddhodge_potential', 'umap_ori_ddhodge_potential'])

In [ ]:
dyn.vf.rank_velocity_genes(adata_labeling, 
                           groups='cell_type', 
                           vkey='velocity_alpha_minus_gamma_s')

In [ ]:
dyn.vf.VectorField(adata_labeling, basis='pca', M=600)
dyn.pl.streamline_plot(adata_labeling, basis='pca', color='cell_type')

In [ ]:
dyn.vf.VectorField(adata_labeling, basis='pca', M=250, lambda_=3, sigma=8)
dyn.vf.acceleration(adata_labeling, basis='pca')
dyn.vf.rank_acceleration_genes(adata_labeling, groups='cell_type')


In [ ]:
dyn.vf.curvature(adata_labeling, basis='pca')
dyn.vf.rank_curvature_genes(adata_labeling, groups='cell_type')


In [ ]:
import numpy as np

dyn.pl.scatters(adata_labeling, 
                x=np.repeat('umap_ori_ddhodge_potential', len(dynamo_revision_genes[:-1])), 
                y=dynamo_revision_genes[:-1], 
                pointsize=0.25, 
                alpha=0.8, 
                layer='M_t', 
                color='cell_type',
                ncols=3, 
                background='white',
               )

In [ ]:
dyn.pl.scatters(adata_labeling, 
                x=np.repeat('umap_ori_ddhodge_potential', len(dynamo_revision_genes[:-1])), 
                y=dynamo_revision_genes[:-1], 
                pointsize=0.25, 
                alpha=0.8, 
                layer='M_n', 
                color='cell_type',
                ncols=3, 
                background='white',
               )

In [ ]:
dyn.pl.scatters(adata_labeling, 
                x=np.repeat('umap_ori_ddhodge_potential', len(dynamo_revision_genes[:-1])), 
                y=dynamo_revision_genes[:-1], 
                pointsize=0.25, 
                alpha=0.8, 
                layer='velocity_alpha_minus_gamma_s', 
                color='cell_type',
                ncols=3, 
                background='white',
               )

In [ ]:
dyn.pl.scatters(adata_labeling, 
                x=np.repeat('umap_ori_ddhodge_potential', len(dynamo_revision_genes[:-1])), 
                y=dynamo_revision_genes[:-1], 
                pointsize=0.25, 
                alpha=0.8, 
                layer='acceleration', 
                color='cell_type',
                ncols=3, 
                background='white',
               )

In [ ]:
Krumsiek_11 = ["GATA2", "GATA1", "ZFPM1", "SPI1", "FLI1", "KLF1", "TAL1", "CEBPA", "GFI1", "JUN", "NAB2"]
# adata.var_names[adata.var.use]
dyn.pl.scatters(adata_labeling,
                x=np.repeat('umap_ori_ddhodge_potential', len(Krumsiek_11)),
                y=Krumsiek_11,
                pointsize=0.25,
                alpha=0.8,
                layer='acceleration',
                color='cell_type',
                ncols=len(Krumsiek_11),
                background='white',
               )

In [ ]:
mt_gene_mask = [gene.startswith('MT-') for gene in adata_labeling.var_names]
adata_labeling.obs['n_counts'] = adata_labeling.layers['total'].sum(1) 
adata_labeling.obs['mt_frac'] = adata_labeling[:, mt_gene_mask].layers['total'].sum(1) 
adata_labeling.obs['mt_frac'] /= adata_labeling.obs['n_counts']

In [ ]:
dyn.pl.streamline_plot(
    adata_labeling,
    color=['mt_frac', 'cell_cycle_phase', 'GATA1'],
    show_legend='best', 
    basis='umap_ori',
    s_kwargs_dict= {"alpha": 0.4}
)

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', color=['PIM1', 'HDC', 'CSF2RB', 'LIF', "Z84723.1"])

# basophil markers

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', color=["FLI1", "KLF1"])

In [ ]:
dyn.pl.streamline_plot(
    adata_labeling, 
    basis='umap_ori', 
    color=["DHFR", "CDK6", "CCNH", "MYC", "CUL2"],
    show_legend='best', 
    s_kwargs_dict= {"alpha": 0.4},
    ncols=5
)

In [ ]:
dyn.pl.streamline_plot(
    adata_labeling, 
    basis='umap_ori',
    color=["DHFR", "CDK6", "CCNH", "MYC", "CUL2"],
    show_legend='best', 
    s_kwargs_dict= {"alpha": 0.4},
    ncols=5,
    layer='X_new',
)

In [ ]:
dyn.pl.streamline_plot(
    adata_labeling, 
    basis='umap_ori',
    color=['NR4A3', 'ZBTB16', 'GATA1'],
    show_legend='best', 
    s_kwargs_dict= {"alpha": 0.4}
)

In [ ]:
dyn.pl.streamline_plot(
    adata_labeling, 
    basis='umap_ori',
    color=['PSMD10', 'DSTYK', 'PLK1', 'HIGD1A', 'SQSTM1', 'HSPA1A'],
    show_legend='best', 
    s_kwargs_dict= {"alpha": 0.4}
)

In [ ]:
dyn.convert2float(adata_labeling, 
                  columns=['gamma', 'gamma_b', 'gamma_k'], 
                  var=True)

In [ ]:
dyn.vf.jacobian(adata_labeling, 
                regulators=adata_labeling.var_names[adata_labeling.var.use_for_pca], 
                effectors=adata_labeling.var_names[adata_labeling.var.use_for_pca])
J_res = dyn.vf.rank_jacobian_genes(adata_labeling, groups='cell_type')

In [ ]:
manual_selection = pd.read_csv('./data/hsc_manual_selection.csv')
manual_selection.head(5)

In [ ]:
adata_labeling.obs['manual_selection'] = manual_selection.loc[:, 'manual_selection'].values

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=['cell_type', 'manual_selection'], basis='umap_ori')

In [ ]:
dyn.vf.rank_jacobian_genes(adata_labeling, mode='switch', groups='manual_selection')

In [ ]:
adata_labeling.uns['switch'].head(10)

In [ ]:
cur_col = adata_labeling.uns['switch']['Mono_baso_curv'][:20]
cur_col

In [ ]:
gene_pairs = cur_col.str.split(" - ", expand=True)

In [ ]:
def parse_int_df(df, self_int=False, genes=None):
    
    df_shape, columns = df.shape, df.columns
    if columns[1].endswith("_values"):
        col_step = 2
    else:
        col_step = 1
    
    res = {}
    if genes is not None: genes_set = set(genes)
    for col in columns[::col_step]:
        cur_col = df[col]
        gene_pairs = cur_col.str.split(" - ", expand=True)
        
        if not self_int: 
            good_int = gene_pairs[0] != gene_pairs[1]
        else:
            good_int = np.ones(df_shape[0], dtype=bool)
            
        if genes is not None: 
            good_int &= np.logical_and([i in genes_set for i in gene_pairs[0]], [i in genes_set for i in gene_pairs[1]])
            
        if col_step == 1:
            res[col] = cur_col.loc[good_int].values
        else:
            res[col] = cur_col.loc[good_int].values
            res[col + '_values'] = df[col + '_values'].loc[good_int].values  

    return pd.DataFrame(res)

res = parse_int_df(adata_labeling.uns['switch'])
res.head(20)

In [ ]:
human_tfs = pd.read_csv('/lab/solexa_weissman/xqiu/proj/Aristotle/data/TFs/Homo_sapiens_TF', sep='\t')
human_tfs

In [ ]:
res = parse_int_df(adata_labeling.uns['switch'], genes=human_tfs['Symbol'].values)

In [ ]:
res.head(20)

In [ ]:
# Ery - Meloid: Gata1 - Spi1; 
# Mega: Fli1 Gata2 ; Ery: Eklf - Klf1
# Gran: Gfi1 ; Macro: Nab2 / Egr / Egr2 
# B cell: Pax5 ; 
# T cell: Gata3; B lymphoid: Ebf1
# erythroid-myeloid: Gata1-Spi1 SPi1-Tal1 Fosb-E2f8 NFia-Spi1 Hes6-Klf4
# Gata3-Spib; Gata3-Aff3; Ebf1-Gata3 

#TF from Paul paper: 
#Cebpe Calr Arid3a Gfi1 Lmo4 Cebpa Pu.1 Cux1 Scand1 Nfkbia Irf8 Id2 Chd3 Cbfa2t3 Etv6 Stat6 Pnrc1 Pbx1 Mef2c Fli1 Elf1 Lmo2 Cited2 Sox4 Runx1 Gata2 Nfe2 Mrb Foxp1 Zfpm1 Hmgb3 Klf1 Gfi1b Tcf3 Pa2g4 Mbd2 Gata1 Phf10 Phb2 Csda E2f4 Cited4 Ccne1  
#Target of Cebpa, Irf8, Klf1, Pu.1 from Paul paper: 
#Cebpa:  Abcb1b, Acot1, C3, Cnpy3, Dhrs7, Dtx4, Edem2, Etfb, Ftl1, Gadd45b, Herpud1, Hp, Il6ra, Lcn2, Lcp1, Lman2, Lrg1, Serpinf1, Snord49b, Tnfrsf1a, Tnfsf14, Trappc1, Trf, Xdh
#Irf8: Abcd1, Aif1, BC017643, Cbl, Ccdc109b, Ccl6, d68, d74, dc42se1, Cdca7, Csf3r, Ctss, Cybb, Cyp4f18, Dtx2, Entpd1, Ercc6l, Erp29, Fastkd3, Ftl1, Gda, Grn, H2-M3, Hexa, Hk3, Ide, Il6ra, Irf5, Lamp1, Ly86, Mtm1, Napsa, Ncf1, Osbpl3, Pdlim2, Phgdh, Pld4, Psmb10, Psmb8, Rgs2, Rod1, Snx10, Spop, Tapbp, Tapbpl, Tm6sf1, Trem2, Trem3, Wipf1, Xdh, Zfp296,
#Irf8 (2): 1100001G20Rik, 4732418C07Rik, 9230105E10Rik, AI504432, Acox3, Actb, Aif1, Akap13, Ap1s3, Arhgap30, Arid5a, Arsk, Ass1, Bcl9l, Bst2, Ccl5, Ccl6, Ccng2, Cd52, Cd74, Cited2, Ctss, Dapp1, Emp3, Erp29, Ets1, Fgd2, Gpr18, Gramd3, H2-K1, Hist4h4, Hk3, Irf5, Lsp1, Ly86, Ncf4, Parp8, Pdlim2, Pecam1, Psap, Psmb8, Ptpn18, S100a10, Shfm1, Sla, Socs1, Tmem50a, Trove2
#Klf1: 2010011I20Rik, 5730469M10Rik, Acsl6, Add2, Ank1, Arhgap23, Blvrb, C330018D20Rik, Ccdc23, Cd24a, Cd82, Ctbp1, Cyth3, Dnajb1, E2f4, Emilin2, Epn1, Fam132a, Fam53b, Gfi1b, Grina, Gypa, Hdgf, Hebp1, Hemgn, Hs6st1, Kel, Klf3, Lmna, Lsm4, Mgst3, Mrpl52, Nudt9, Pla2g16, Ppap2a, Prdx2, Prkab1, Samd14, Sepw1, Snx3, Srxn1, Ucp2, Vamp5, Zcchc6,
#Pu.1: 0910001L09Rik, 2310014H01Rik, 4632428N05Rik, Abcd1, Anxa2, Ap3s1, Arid3a, Arpc1b, Arpc2, Bola2, Calm1, Cfl1, Chd7, Cited2, Csf1r, Csf3r, Ctsc, Dnajb11, Dnajc3, Edem1, Emilin1, Eml4, Ero1lb, Fes, Fgd2, G6pc3, Gsr, H13, H47, Hexa, Hspa5, Hyou1, Ifngr2, Igfbp4, Il21r, Irf1, Kdelr2, Krtcap2, Lamp1, Lcp1, Lta4h, Ltb4r1, Man2b1, Manf, Myl12b, Myo1f, Napsa, Ncf2, Ncf4, Pdia3, Pkm2, Pld4, Prtn3, Psmb8, Pstpip1, Ptgs1, Ptpn18, Renbp, Sec61b, Sh2b2, Sirpa, Ssr4, Tkt, Tmbim6, Tmed9, Tnfrsf1a, Tyrobp, Unc93b1, Vav1
# 'Nfe2', 'Gata2', 'SCL', 'Gfi1', 'Gfi1b'
# known megakaryocytic genes (e.g. Cd9, Lox, Vwf, Nfib,Cd61, Tgfβ1) and a cluster with several red edges comprising 
# known erythroid genes (e.g. Cd36, Klf1, Lef1, Cnrip1, Tmod1, Ank1, Dhrs3) 
# # 'Gata1_Zfpm1', 'Gata1_Gfi1b', 'Gfi1b_Gata1', 'Gata1_Tcf3', 'Gata1_Klf1', 'Klf1_Zfpm1'
# RBC: GATA1, FOG1, GFI1B, EKLF, 
# Meg: GATA1, GATA2, FOG1, GFI1B, FLI1, NFE2
# MAST: GATA2, GATA1 
# Eosinophil: GATA1, CEBPA, CEBPE
# B: E2A, EBF, PAX5, BCL11A, EVI9, XBP1
# T: IKAROS, PU1, NOTCH, TCF1, GATA3
# LT-HSC: RUNX1, SCL, TAL1, LMO2, MII, TEL, BMI1, GFI1, GATA2 

In [ ]:
columns = res.columns
if columns[1].endswith("_values"):
    col_step = 2
else:
    col_step = 1

for col in columns[::col_step]:
    cur_col = res[col][:5]
    gene_pairs = cur_col.str.split(" - ", expand=True)
    
    for i, gene in gene_pairs.iterrows(): 
        dyn.pl.jacobian(adata_labeling, x=gene[0], y=gene[1], layer='M_t', regulators=gene[0], effectors=gene[1])
        dyn.pl.scatters(adata_labeling, color=gene, basis='umap_ori')

In [ ]:
gene[1][0]

In [ ]:
np.logical_and([i in set(["HBB", "PLEK"]) for i in gene_pairs[0]], [i in set(["HBB", "PLEK"]) for i in gene_pairs[0]]) & np.logical_and([i in set(["HBB", "PLEK"]) for i in gene_pairs[0]], [i in set(["HBB", "PLEK"]) for i in gene_pairs[0]])

In [ ]:
dyn.vf.rank_curvature_genes(adata_labeling, groups='manual_selection')
dyn.vf.rank_divergence_genes(adata_labeling, groups='manual_selection')


In [ ]:
high_curvature_genes = (np.unique(adata_labeling.uns['rank_abs_curvature'].iloc[:, :-1].head(100).values))
len(high_curvature_genes)

In [ ]:
remove_high_curvature_genes = adata_labeling.var_names[adata_labeling.var.use_for_pca].difference(high_curvature_genes)

len(remove_high_curvature_genes)

In [ ]:
random_genes = np.random.choice(adata_labeling.var_names[adata_labeling.var.use_for_pca], 250)

In [ ]:
adata_labeling.obsm['X_umap_ori_test'] = adata_labeling.obsm['X_umap_ori'].copy()
dyn.tl.cell_velocities(adata_labeling,
                       X=adata_labeling[:, remove_high_curvature_genes].layers['M_t'],
                       V=adata_labeling[:, remove_high_curvature_genes].layers['velocity_alpha_minus_gamma_s'],
                       basis='umap_ori_test',
                       method='cosine',
                       enforce=True,
                       )
dyn.pl.streamline_plot(adata_labeling, color='cell_type', basis='umap_ori_test')

In [ ]:
adata_labeling

In [ ]:
dyn.vf.divergence(adata_labeling, basis='pca')
dyn.vf.curvature(adata_labeling, basis='umap_ori')

dyn.vf.acceleration(adata_labeling, basis='umap_ori')
dyn.vf.curvature(adata_labeling, basis='umap_ori')


dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori',
                       ncols=4,
                       color=['curvature_pca', 'curvature_umap_ori', 'acceleration_pca', 'acceleration_umap_ori'])

In [ ]:
dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori', 
                       cmap='bwr', 
                       s_kwargs_dict={"sym_c": True}, 
                       ncols=5,
                       color=['curl_umap_ori', 'divergence_pca', 'divergence_umap_ori', 'pca_ddhodge_div', "umap_ori_ddhodge_div"])

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', cmap='bwr', s_kwargs_dict={"sym_c": True},
                       color=['curvature_pca', 'curvature_umap_ori', 'acceleration_pca', 'curl_umap_ori', 'divergence_pca', 
                                              'divergence_umap_ori', 'pca_ddhodge_div', "umap_ori_ddhodge_div"])

In [ ]:
adata_labeling.obs['acceleration_pca'].min(), adata_labeling.obs['acceleration_pca'].max()

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori', frontier=True, sort='abs',
                       color=['curvature_pca', 'acceleration_pca'])

In [ ]:
adata_labeling.uns['rank_abs_velocity_alpha_minus_gamma_s'].head(10)

In [ ]:
# dyn.pl.jacobian(adata_labeling, x="HBB", y="PLEK", layer='M_t')
dyn.pl.jacobian(adata_labeling, x="HBB", y="PLEK", layer='M_t', regulators=['HBB'], effectors=['PLEK'])

In [ ]:
dyn.pl.jacobian(adata_labeling, x="FGFR1", y="HBG2", layer='M_t', regulators=['FGFR1'], effectors=['HBG2'])

In [ ]:
dyn.pl.jacobian(adata_labeling, x="FCER1A", y="GSTP1", layer='M_t', regulators=['FCER1A'], effectors=['GSTP1'])

In [ ]:
from dynamo.plot.utils import _matplotlib_points

In [ ]:
# ?_matplotlib_points

In [ ]:
type(adata_labeling[:, "FCER1A"].layers["M_t"].A.flatten())

In [ ]:
dyn.pl.scatters(adata_labeling, color='cell_type',
                x=adata_labeling[:, "FCER1A"].layers["M_t"].A.flatten(), 
                y=adata_labeling[:, "GSTP1"].layers["velocity_alpha_minus_gamma_s"].A.flatten())

In [ ]:
adata_labeling[:, "GSTP1"].layers["velocity_alpha_minus_gamma_s"].A.flatten()

In [ ]:
adata_labeling.uns['rank_abs_velocity_alpha_minus_gamma_s'].head(10)

In [ ]:
adata_labeling.uns['rank_velocity_alpha_minus_gamma_s'].head(10)

In [ ]:
adata_labeling.uns['rank_abs_acceleration'].head(50)

In [ ]:
dyn.pl.scatters(adata_labeling, 
                color='AL035078.1', 
                basis='umap_ori')

In [ ]:
dyn.pl.scatters(adata_labeling, color='AL035078.1', basis='umap_ori', layer='acceleration', cmap='bwr', sym_c=True, use_smoothed=False)

In [ ]:
dyn.pl.scatters(adata_labeling, 
                color=dynamo_revision_genes[:-1], 
                basis='umap_ori')

In [ ]:
dyn.pl.scatters(adata_labeling, 
                color=dynamo_revision_genes[:-1], 
                basis='umap_ori', 
                layer='acceleration', 
                cmap='bwr', 
                sym_c=True, 
                use_smoothed=False)

In [ ]:
dyn.pl.phase_portraits(adata_labeling, 
                       color='cell_type',
                       genes=dynamo_revision_genes[:-1],
                       ekey='M_t',
                       vkey='velocity_alpha_minus_gamma_s',
                       basis='umap_ori'
                      )

In [ ]:
adata_labeling

In [ ]:
dyn.pl.scatters(adata_labeling, 
                color=dynamo_revision_genes[:-1], 
                basis='umap_ori', 
                layer='velocity_alpha_minus_gamma_s', 
                cmap='bwr', 
                sym_c=True, 
                use_smoothed=False)

In [ ]:
sc.tl.rank_genes_groups(adata_labeling, 'cell_type', method='t-test', layer='velocity_alpha_minus_gamma_s', use_raw=False)
sc.set_figure_params(dpi=150)
sc.pl.rank_genes_groups(adata_labeling, n_genes=25, sharey=False)


In [ ]:
sc.tl.rank_genes_groups(adata_labeling, 'cell_type', method='t-test', layer='acceleration', use_raw=False)
sc.set_figure_params(dpi=150)
sc.pl.rank_genes_groups(adata_labeling, n_genes=25, sharey=False)

In [ ]:
sc.tl.rank_genes_groups(adata_labeling, 'cell_type', method='t-test', layer='curvature', use_raw=False)
sc.set_figure_params(dpi=150)
sc.pl.rank_genes_groups(adata_labeling, n_genes=25, sharey=False)

In [ ]:
gene_list = sc.get.rank_genes_groups_df(adata_labeling, group="MEP-like").head(20)['names'].to_list()
valid_gene_list = list(np.array(gene_list)[~ np.array([True if i.startswith("MT") else False for i in gene_list])])

res = dyn.ext.enrichr(valid_gene_list, organism='human', cutoff=0.1)

res.res2d.head(20)

In [ ]:
two_group_M_n = dyn.tl.two_groups_degs(adata_labeling, 
                       group='cell_type', 
                       genes=adata_labeling.var_names[adata_labeling.var.use_for_pca], 
                       layer='acceleration',
                       test_group='Neu', 
                       control_groups='Mon')

In [ ]:
two_group_M_n.sort_values('log2_fc', ascending=False).head(20)

In [ ]:
two_group_M_n = dyn.tl.two_groups_degs(adata_labeling, 
                       group='cell_type', 
                       genes=adata_labeling.var_names[adata_labeling.var.use_for_pca], 
                       layer='acceleration',
                       test_group='Neu', 
                       control_groups='Mon')
two_group_M_n.sort_values('log2_fc', ascending=False).head(20)

In [ ]:
# two_group_M_n = dyn.tl.two_groups_degs(adata_labeling, 
#                        group='cell_type', 
#                        genes=adata_labeling.var_names[adata_labeling.var.use_for_pca], 
#                        layer='curvature',
#                        test_group='Neu', 
#                        control_groups='Mon')
# two_group_M_n.sort_values('diff_ratio_pos', ascending=False).head(20)

In [ ]:
res = dyn.ext.enrichr(two_group_M_n.sort_values('specificity', ascending=False)['gene'][:50], organism='human', cutoff=0.1)

res.res2d.head(20)

In [ ]:
adata_labeling

In [ ]:
dyn.vf.VectorField(adata_labeling, basis='umap', M=1000)
dyn.vf.curvature(adata_labeling,  basis='umap')

In [ ]:
dyn.vf.curvature(adata_labeling,  basis='umap_ori')

In [ ]:
feature_df = np.hstack((adata_labeling.obs.loc[:, ["curvature_pca", "curvature_umap_ori"]].values, adata_labeling.obsm['X_umap_ori']))

feature_df[:5, :]

# identify cell groups with high curvature values and then run curvature ranking for those groups. 
This makes more sense

In [ ]:
from anndata import AnnData
feature_adata = AnnData(feature_df)
feature_adata.obsm['X_pca'] = feature_df
feature_adata.obsm['X_umap'] = feature_df[:, 2:]

dyn.tl.louvain(feature_adata, obsm_key="X_pca")
dyn.tl.leiden(feature_adata, obsm_key="X_pca")
# dyn.tl.infomap(feature_adata, obsm_key="X_pca")

dyn.pl.umap(feature_adata, color=['louvain', 'leiden']) # , 'infomap'

In [ ]:
dyn.pl.streamline_plot(adata_labeling, basis='umap_ori')

In [ ]:
adata_labeling.obs['curv_leiden'], adata_labeling.obs['curv_louvain'] = feature_adata.obs['leiden'].values.astype(str).copy(), feature_adata.obs['louvain'].values.astype(str).copy()
adata_labeling.obs['curv_leiden'], adata_labeling.obs['curv_louvain']


In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=['curv_leiden', 'curv_louvain', 'cell_type'], basis='umap_ori') # , 'infomap'

In [ ]:
adata_labeling.uns['rank_curvature'].head(10)

In [ ]:
adata_labeling.uns['rank_abs_curvature'].head(10)

In [ ]:
Meg_genes = ['CD9', 'LOX', 'MPL', 'VWF', 'NF1B', 'CD61', 'CD41']
Ery_genes = ['MYB', 'TMOD1', 'LEF1', 'KLF1', 'CNRIP', 'ANK1']
Mep_genes = ['CD44', 'KIT', 'DHRS3', 'GATA1', 'CD36']
other_genes = ['IATGB3', 'CD9', 'MPL', 'CD47', 'ANK1', 'ITGA2B', 'TGFB1', 'MYB']

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=list(adata_labeling.var_names.intersection(Meg_genes)) + ['cell_type'], basis='umap_ori')

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=list(adata_labeling.var_names.intersection(Ery_genes)) + ['cell_type'], basis='umap_ori')

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=list(adata_labeling.var_names.intersection(Mep_genes)) + ['cell_type'], basis='umap_ori')

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=list(adata_labeling.var_names.intersection(other_genes)) + ['cell_type'], basis='umap_ori')

In [ ]:
all_valid_genes = ["ADD2", "ANK1", "B2M", "BDH1", "BIM", "BCL2", "BMP2", "CCDC124", "CCND1", "BCL1", "CD105", "ENG", "CD110", "MPL", "CD117", "KIT", "CD123", 
                   "IL3RA", "CD135", "FLT3", "CD235a", "GYPA", "CD34", "CD36", "CD38", "CD41", "ITGA2B", "CD42", "GP1BA", "CD44", "CD47", "CD61", "ITGB3", 
                   "CD71", "TFRC", "CD9", "CD90", "THY1", "CDK4", "CDK6", "CEBPA", "CISH", "CLU", "CNRIP1", "CSF3R", "CTNNB1 ", "CXCR4", "DDB1", "DHRS3", 
                   "ECT2", "EIF4EBP1", "EPOR", "ETS1", "FAS", "FHL1", "FLI1", "FOG1", "ZFPM1", "FOXO3", "GABPA", "GADD45A", "GADD45B", "GAPDH ", "GATA1", 
                   "GATA2", "GFI1B", "HIF1A", "HIF2A", "EPAS1", "HPRT1", "JAK2", "KLF1", "LEF1", "LOX", "LSD1", "KDM1A", "MCL1", "MEIS1", "MPO", "MYB", 
                   "NFIB", "NOTCH1", "P27KIP1", "CDKN1B", "PAX5", "PF4", "PFKP", "PIM1", "PNMT", "RGS18", "RUNX1", "SDPR", "SELP ", "SH2D2A", "SOCS3", 
                   "SPI1", "SRD5A3", "TAL1", "TGFB1", "TGFB2", "TMEM180", "MFSD13A", "TMOD1", "TUBB2A", "VWF"]

# do in silico perturbation 

# res = dyn.ext.enrichr(list(adata_labeling.uns['rank_abs_velocity_T']['0'][:50]), organism='human')

# res.res2d.head(20)

gene2celltype['MPO']

res = dyn.ext.enrichr(list(adata_labeling.uns['rank_abs_acceleration']['0'][:50]), organism='human')

res.res2d.head(20)

res = dyn.ext.enrichr(list(adata_labeling.uns['rank_abs_curvature']['0'][:50]), organism='human')

res.res2d.head(20)

In [ ]:
set(adata_labeling.uns['rank_abs_acceleration']['Meg']).intersection(Mep_genes)

In [ ]:
set(adata_labeling.uns['rank_abs_acceleration']['HSC']).intersection(other_genes)

In [ ]:
list(adata_labeling.uns['rank_abs_acceleration']['Meg']).index('VWF'), list(adata_labeling.uns['rank_abs_acceleration']['Meg']).index('KLF1'), list(adata_labeling.uns['rank_abs_acceleration']['Meg']).index('GATA1')

In [ ]:
adata_labeling.uns['rank_abs_acceleration']['Meg'][:50]

In [ ]:
res = dyn.ext.enrichr(list(adata_labeling.uns['rank_abs_acceleration']['Meg'][:50]), organism='human')

res.res2d.head(20)

In [ ]:
adata_labeling.uns['rank_abs_curvature'].head(10)

In [ ]:
dyn.vf.rank_acceleration_genes(adata_labeling, groups='cell_type', normalize=True)
adata_labeling.uns['rank_abs_acceleration'].head(10)

# Jacobian analysis

In [ ]:
full_reg_rank = dyn.vf.rank_jacobian_genes(adata_labeling,
                                           groups='cell_type',
                                           mode="full_reg",
                                           abs=True,
                                           output_values=True)

In [ ]:
full_eff_rank = dyn.vf.rank_jacobian_genes(adata_labeling,
                                           groups='cell_type',
                                           mode="full_eff",
                                           abs=True,
                                           output_values=True)

In [ ]:
full_reg_rank, full_eff_rank = adata_labeling.uns['full_reg'], adata_labeling.uns['full_eff']

In [ ]:
full_reg_rank["HSC"].columns.intersection(['GATA1', 'ZFPM1', 'GATA2', 'MIR150', 'MIR451', 'ZBTB7A',  'TAL1', 'KLF1', "FLI1"])

In [ ]:
full_reg_rank["Meg"].loc[:, ['GATA1', 'GATA1_values', 'KLF1', 'KLF1_values']].head(30)

In [ ]:
full_eff_rank["Meg"].loc[:, ['GATA1', 'GATA1_values', 'KLF1', 'KLF1_values']].head(30)

In [ ]:
set(full_eff_rank["Meg"]['GATA1'][:20]).intersection(full_eff_rank["Meg"]['KLF1'][:20])

In [ ]:
dyn.pl.jacobian(adata_labeling, 
                regulators=['GATA1', 'KLF1'], 
                effectors=['C6', 'TIAM1'],
                basis='umap_ori'
               )

In [ ]:
dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori',
                       color=adata_labeling.var_names.intersection(['GATA1', 'ZFPM1', 'GATA2', 'MIR150', 'MIR451', 'ZBTB7A',  'TAL1', 'KLF1', "FLI1"]))

In [ ]:
dyn.pl.streamline_plot(adata_labeling, 
                       basis='umap_ori', 
                       color=adata_labeling.var_names.intersection(['GATA1', 'ZFPM1', 'GATA2', 'MIR150', 'MIR451', 'ZBTB7A',  'TAL1', 'KLF1', "FLI1"]), 
                       layer='X_new')

In [ ]:
adata_labeling.var_names.intersection(['KLF1', "FLI1"])

In [ ]:
reg_rank = dyn.vf.rank_jacobian_genes(adata_labeling,
                                      groups='cell_type',
                                      mode="reg",
                                      abs=True,
                                      normalize=True,
                                      return_df=True,
                                      output_values=True)
reg_rank

In [ ]:
eff_rank = dyn.vf.rank_jacobian_genes(adata_labeling,
                                      groups='cell_type',
                                      mode="eff",
                                      abs=True,
                                      normalize=True,
                                      return_df=True,
                                      output_values=True)
reg_rank

In [ ]:
int_rank = dyn.vf.rank_jacobian_genes(adata_labeling,
                                      groups='cell_type',
                                      mode="int",
                                      abs=True,
                                      normalize=True,
                                      return_df=True,
                                      output_values=True)
int_rank

In [ ]:
reg_rank['Meg'].head(20)

In [ ]:
eff_rank['Meg'].head(20)

In [ ]:
int_rank['Meg'].head(20)

In [ ]:
dyn.pl.jacobian(adata_labeling, regulators=['KLF1', 'GATA1'], effectors=['KLF1', 'GATA1'])

In [ ]:
dyn.pl.jacobian(adata_labeling, regulators=['TIAM1', 'RORA'], effectors=['TIAM1', 'RORA'])

# in silico-perturbation

dyn.pd.perturbation(adata_labeling, genes=['GATA1'], expression=[0])

dyn.pl.streamline_plot(adata_labeling, basis='umap_perturbation', color=['GATA1'])

dyn.pd.perturbation(adata_labeling, genes=['KLF1'], expression=[0])

dyn.pd.perturbation(adata_labeling, genes=adata_labeling.var_names[adata_labeling.var.use_for_pca].intersection(["TAL1", "LMO2"]), expression=[100])

dyn.pl.streamline_plot(adata_labeling, basis='umap_perturbation', color=adata_labeling.var_names[adata_labeling.var.use_for_pca].intersection(['GATA1', "TAL1", "LMO2"]))

from dynamo.vectorfield.scVectorField import vector_field_function_knockout
from dynamo.vectorfield import SvcVectorfield
vf = SvcVectorfield()
vf.from_adata(adata_labeling, basis='pca', vf_key='VecFld')

KO_genes = ['GATA1'] # , 'Ins2'
vf_perturb = vector_field_function_knockout(adata_labeling, vf, KO_genes)

adata_labeling.obsm['velocity_pca_perturb'] = vf_perturb.get_V()
adata_labeling.obsm['X_pca_perturb'] = adata_labeling.obsm['X_pca'].copy()

# get the perturbed vector field on umap space
adata_labeling.obsm['X_umap_perturb'] = adata_labeling.obsm['X_umap'].copy()
dyn.tl.cell_velocities(adata_labeling,
                       X=adata_labeling.obsm['X_pca'],
                       V=adata_labeling.obsm['velocity_pca_perturb'], #- adata.obsm['velocity_pca'],
                       basis='umap_perturb', enforce=True
                       )
# adata_labeling.obsm["velocity_umap_perturb"] -= adata_labeling.obsm["velocity_umap"]
dyn.pl.streamline_plot(adata_labeling, basis='umap_perturb', color='GATA1')

KO_genes = ['KLF1'] # , 'Ins2'
vf_perturb = vector_field_function_knockout(adata_labeling, vf, KO_genes)

adata_labeling.obsm['velocity_pca_perturb'] = vf_perturb.get_V()
adata_labeling.obsm['X_pca_perturb'] = adata_labeling.obsm['X_pca'].copy()

# get the perturbed vector field on umap space
adata_labeling.obsm['X_umap_perturb'] = adata_labeling.obsm['X_umap'].copy()
dyn.tl.cell_velocities(adata_labeling,
                       X=adata_labeling.obsm['X_pca'],
                       V=adata_labeling.obsm['velocity_pca_perturb'], #- adata.obsm['velocity_pca'],
                       basis='umap_perturb', enforce=True
                       )
# adata_labeling.obsm["velocity_umap_perturb"] -= adata_labeling.obsm["velocity_umap"]
dyn.pl.streamline_plot(adata_labeling, basis='umap_perturb', color='KLF1')

dyn.pl.streamline_plot(adata_labeling, basis='umap', color =['CD34', "FLT3", "GYPA"])

dyn.pl.jacobian(adata_labeling, regulators=['GATA1', 'KLF1'], effectors=['C6', 'TIAM1'])

adata_labeling.uns['rank_abs_velocity_T'].head(10)

dyn.pl.scatters(adata_dyn, color=['TUBA1B', 'AC243919.1', 'ENO1', 'MCM7'], layer='acceleration')

dyn.pl.scatters(adata_dyn, color=['TUBA1B', 'AC243919.1', 'ENO1', 'MCM7'])

dyn.pl.phase_portraits(adata_labeling, genes=Vijay_genes + chen_genes + klein_genes + paul_genes, color='leiden')

adata_labeling.var.g

dyn.pl.phase_portraits(adata_labeling, genes=chen_paper_genes, color='leiden')

########### splicing data
adata_dyn = dyn.read('./adata_dyn.h5ad')
adata_filtered = dyn.read('./adata_filtered_hsc.h5ad')

del adata_dyn.layers['new'], adata_dyn.layers['total']
adata_dyn.obs['time'] = 3
adata_dyn.X = adata_dyn.layers['spliced'].copy()

adata_splicing = adata_dyn.copy()

dyn.pp.recipe_monocle(adata_splicing,
                      genes_to_use=adata_filtered.var.gene_name,
                      n_top_genes=adata_filtered.n_vars,
                      # fg_kwargs={"min_avg_exp_s": 1,
                      #            "min_cell_s": 100,
                      #            "shared_count": 50}
                     )

adata_splicing.obs['leiden_ori'] = adata_filtered.obs['leiden']
dyn.tl.dynamics(adata_splicing, group='leiden_ori')
dyn.tl.reduceDimension(adata_splicing)
dyn.tl.cell_velocities(adata_splicing,
                       transition_genes=adata_splicing.var_names[adata_splicing.var.use_for_pca],
                       enforce=True,
                       method='cosine',
                       ) # , transition_genes=adata_filtered.var_names[adata_filtered.var.use_for_pca], enforce=True
dyn.tl.leiden(adata_splicing)

Vijay_genes = ["GFI1", "RUNX1", "IRF4", "IRF8", "SPI1", 'MPO']
dyn.pl.streamline_plot(adata_splicing, color=Vijay_genes + ['leiden'])

chen_paper_genes = ['PLEK', 'HBB', 'MPO', 'SPIB', 'CD79A', 'DNTT', 'CD34', 'CD164']
dyn.pl.streamline_plot(
    adata_splicing,
    color=adata_splicing.var_names.intersection(chen_paper_genes),
    ncols=4,
)

########### splicing data
adata_dyn = dyn.read('./adata_dyn.h5ad')
adata_filtered = dyn.read('./adata_filtered_hsc.h5ad')

del adata_dyn.layers['new'], adata_dyn.layers['total']
adata_dyn.obs['time'] = 3
adata_dyn.X = adata_dyn.layers['spliced'].copy()

adata_splicing = adata_dyn.copy()

dyn.pp.recipe_monocle(adata_splicing,
                      genes_to_use=adata_filtered.var.gene_name,
                      n_top_genes=adata_filtered.n_vars,
                      # fg_kwargs={"min_avg_exp_s": 1,
                      #            "min_cell_s": 100,
                      #            "shared_count": 50}
                     )

adata_splicing.obs['leiden_ori'] = adata_filtered.obs['leiden']
dyn.tl.dynamics(adata_splicing)
dyn.tl.reduceDimension(adata_splicing)
dyn.tl.cell_velocities(adata_splicing,
                       transition_genes=adata_splicing.var_names[adata_splicing.var.use_for_pca],
                       enforce=True,
                       method='cosine',
                       ) # , transition_genes=adata_filtered.var_names[adata_filtered.var.use_for_pca], enforce=True
dyn.tl.leiden(adata_splicing)

Vijay_genes = ["GFI1", "RUNX1", "IRF4", "IRF8", "SPI1", 'MPO']
dyn.pl.streamline_plot(adata_splicing, color=Vijay_genes + ['leiden'])

chen_paper_genes = ['PLEK', 'HBB', 'MPO', 'SPIB', 'CD79A', 'DNTT', 'CD34', 'CD164']
dyn.pl.streamline_plot(
    adata_splicing,
    color=adata_splicing.var_names.intersection(chen_paper_genes),
    ncols=4,
)

dyn.pl.phase_portraits(adata_splicing, genes=Vijay_genes + chen_genes + klein_genes + paul_genes, color='leiden')


import warnings
warnings.filterwarnings('ignore')

dyn.pl.scatters(adata_labeling, 
                basis=adata_labeling.var_names.intersection(Vijay_genes + chen_genes + klein_genes + paul_genes), 
                x='X_total', 
                y='X_new',
                color='leiden',
                use_smoothed=False,
                pointsize=0.2,
                alpha=0.6,
               )


dyn.pl.scatters(adata_splicing, 
                basis=adata_labeling.var_names.intersection(Vijay_genes + chen_genes + klein_genes + paul_genes), 
                x='X_spliced', 
                y='X_unspliced',
                color='leiden',
                use_smoothed=False, 
                pointsize=0.2,
                alpha=0.6,
               )


dyn.tl.cell_velocities(adata_splicing,
                       transition_genes=adata_splicing.var_names[adata_splicing.var.use_for_pca],
                       enforce=True,
                       method='cosine',
                       ) # , transition_genes=adata_filtered.var_names[adata_filtered.var.use_for_pca], enforce=True
dyn.tl.leiden(adata_splicing)

Vijay_genes = ["GFI1", "RUNX1", "IRF4", "IRF8", "SPI1", 'MPO']
dyn.pl.streamline_plot(adata_splicing, color=Vijay_genes + ['leiden'])

chen_paper_genes = ['PLEK', 'HBB', 'MPO', 'SPIB', 'CD79A', 'DNTT', 'CD34', 'CD164']
dyn.pl.streamline_plot(
    adata_splicing,
    color=adata_splicing.var_names.intersection(chen_paper_genes),
    ncols=4,
)

dyn.pl.phase_portraits(adata_splicing, genes=chen_genes, color='leiden')



dyn.tl.confident_cell_velocities(adata_dyn,
                                 lineage_dict={"0": ['2', '3']},
                                 group='leiden',
                                 vkey='velocity_T',
                                 ekey='M_t'
                                 )

dyn.pl.phase_portraits(adata_dyn, genes=adata_dyn.var.avg_confidence.sort_values(ascending=False).index[:5])
dyn.pl.phase_portraits(adata_dyn, genes=adata_dyn.var.avg_confidence.sort_values(ascending=False).index[:5], color='leiden')
adata_dyn.var['mu'] = adata_dyn.layers['X_total'].mean(0).A1
dyn.pl.phase_portraits(adata_dyn, genes=adata_dyn.var.query("mu > 1").avg_confidence.sort_values(ascending=False).index[:5], color='leiden')

# save data

In [ ]:
adata_labeling.write_h5ad('./adata_labeling_for_lap_hsc_network.h5ad')

In [ ]:
adata_labeling[:, adata_labeling.var.use_for_pca].write_h5ad('./adata_labeling_for_lap_hsc_network_subset.h5ad')

In [ ]:
adata_labeling[:, Krumsiek_11].var.use_for_pca.sum(), len(Krumsiek_11)

In [ ]:
adata_labeling[:, Krumsiek_11].write_h5ad('./adata_labeling_for_lap_hsc_network_only_hsc_network_genes.h5ad')

In [ ]:
adata_labeling_11_genes = adata_labeling[:, Krumsiek_11]

In [ ]:
adata_labeling_11_genes

In [ ]:
for i in ['total', 'new', 'M_tt', 'M_tn', 'M_nn', 'velocity_N', 'velocity_T', 'velocity_U', 'j_delta_x_perturbation']:
    del adata_labeling_11_genes.layers[i]

In [ ]:
for i in ['moments_con', 'distances', 'connectivities', 'cosine_transition_matrix', 'X_umap_ori_distances', 'X_umap_ori_connectivities', 'pca_ddhodge', 'umap_ori_ddhodge', 'fp_transition_rate', 'perturbation_transition_matrix']:
    del adata_labeling_11_genes.obsp[i]

In [ ]:
for i in ['full_reg', 'full_eff']:
    del adata_labeling_11_genes.uns[i]

In [ ]:
adata_labeling_11_genes
for i in ['full reg', 'switch', 'rank_div_gene_jacobian_pca', 'grid_velocity_umap_ori_test', 'VecFld_umap', 'grid_velocity_umap_perturbation', 'grid_velocity_umap_ori_perturbation']:
    del adata_labeling_11_genes.uns[i]

In [ ]:
for i in ['rank_velocity_alpha_minus_gamma_s', 'rank_abs_velocity_alpha_minus_gamma_s', 'rank_acceleration', 'rank_abs_acceleration', 'rank_curvature', 'rank_abs_curvature', 'jacobian_pca']:
    del adata_labeling_11_genes.uns[i]

In [ ]:
adata_labeling_11_genes.write_h5ad('./adata_labeling_for_lap_hsc_network_only_hsc_network_genes.h5ad')

In [ ]:
adata_labeling_11_genes


In [ ]:
Krumsiek_11_net = pd.read_csv(
    'https://raw.githubusercontent.com/theislab/scanpy/master/scanpy/sim_models/krumsiek11.txt', sep=r'\s+',
    skiprows=np.arange(24))
Krumsiek_11_net = Krumsiek_11_net.replace(
    {
        "Pu.1": "Spi1",
         "Fog1": "ZFPM1",
         "EKLF": "KLF1",
         "SCL": "TAL1",
         "cJun": "Jun",
         "EgrNab": "NAB2"
                     })

adata_labeling_11_genes.layers['M_s'], adata_labeling_11_genes.layers['velocity_S'] = adta_labeling.layers['M_t'].copy(), adata_labeling_11_genes.layers['velocity_alpha_minus_gamma_s'].copy()
for ind, i in Krumsiek_11_net.iterrows():
    if i[0] != i[1]:
        print(dyn.pl.streamline_plot(adata_labeling_11_genes, 
                                     color='cell_type', 
                                     x=i[1].upper(),  # 1 is source 2 is target
                                     y=i[0].upper(), 
                                     ekey='M_t', 
                                     vkey='velocity_alpha_minus_gamma_s'))


In [ ]:
sns.scatterplot(*adata_labeling_11_genes[:, ['SPI1', 'GATA1']].layers["M_s"].A.copy().T)

In [ ]:
adata_labeling_11_genes

In [ ]:
adata_labeling_11_genes.layers['M_s'] = adata_labeling[:, adata_labeling_11_genes.var_names].layers['M_t'].copy()
dyn.pl.jacobian(adata_labeling, x='SPI1', y='GATA1', layer='M_t',
                regulators='SPI1',
                effectors='GATA1')
dyn.pl.streamline_plot(adata_labeling_11_genes, x='SPI1', y='GATA1')

In [ ]:
dyn.pl.jacobian(adata_labeling, 
                x='GATA1', 
                y='GATA2', 
                layer='M_t',
                regulators='GATA1',
                effectors='GATA2')

adata_labeling_11_genes.layers['M_s'] = adata_labeling[:, adata_labeling_11_genes.var_names].layers['M_t'].copy()
dyn.pl.streamline_plot(adata_labeling_11_genes, x='GATA1', y='GATA2')

In [ ]:
dyn.pl.jacobian(adata_labeling, x='FLI1', y='KLF1', layer='M_t',
                regulators='FLI1',
                effectors='KLF1')
dyn.pl.streamline_plot(adata_labeling_11_genes, x='FLI1', y='KLF1')

In [ ]:
dyn.pl.jacobian(adata_labeling, x='GATA1', y='GATA2', layer='M_t',
                regulators='GATA1',
                effectors='GATA2')
dyn.pl.streamline_plot(adata_labeling_11_genes, x='GATA1', y='GATA2')

In [ ]:
for ind, i in Krumsiek_11_net.iterrows():
    if i[0] != i[1]:
        print(dyn.pl.streamline_plot(adata_labeling_11_genes, 
                                     color='cell_type', 
                                     x=i[1].upper(),  # i[1] is source; i[0] is target
                                     y=i[0].upper(), 
                                     ekey='M_t', 
                                     vkey='velocity_alpha_minus_gamma_s'))
        dyn.pl.jacobian(adata_labeling, x=i[1], y=i[0], basis='M_t',
                regulators=i[1], 
                effectors=i[0],
                    
                basis='umap_ori',
                save_show_or_return='save',
                save_kwargs={"prefix": 'Krumsiek_11_jacobian_pca_vecfld_' + str(i[1]) + "->" + str(i[0])}
               )
        dyn.pl.jacobian(adata_labeling, 
                regulators=i[1], 
                effectors=i[0],
                basis='umap_ori',
                save_show_or_return='save',
                save_kwargs={"prefix": 'Krumsiek_11_jacobian_pca_vecfld_' + str(i[1]) + "->" + str(i[0])}
               )

In [ ]:
dyn.pl.jacobian(adata_labeling, 
                regulators=adata_labeling.var_names[adata_labeling.var.use_for_pca].intersection(Krumsiek_11), 
                effectors=adata_labeling.var_names[adata_labeling.var.use_for_pca].intersection(Krumsiek_11),
                basis='umap_ori',
                save_show_or_return='save',
                save_kwargs={"prefix": 'Krumsiek_11_jacobian_pca_vecfld'}
               )

In [ ]:
from dynamo.vectorfield.utils import intersect_sources_targets

def get_jacobian(adata,
                 regulators,
                 effectors,
                 jkey: str = "jacobian",
                 j_basis: str = "pca",
                 ):

    regulators, effectors = (
        list(np.unique(regulators)) if regulators is not None else None,
        list(np.unique(effectors)) if effectors is not None else None,
    )

    Jacobian_ = jkey if j_basis is None else jkey + "_" + j_basis
    Der, cell_indx, jacobian_gene, regulators_, effectors_ = (
        adata.uns[Jacobian_].get(jkey.split("_")[-1]),
        adata.uns[Jacobian_].get("cell_idx"),
        adata.uns[Jacobian_].get(jkey.split("_")[-1] + "_gene"),
        adata.uns[Jacobian_].get("regulators"),
        adata.uns[Jacobian_].get("effectors"),
    )

    adata_ = adata[cell_indx, :]

    if regulators is None and effectors is not None:
        regulators = effectors
    elif effectors is None and regulators is not None:
        effectors = regulators
    # test the simulation data here
    if regulators_ is None or effectors_ is None:
        if Der.shape[0] != adata_.n_vars:
            source_genes = [j_basis + "_" + str(i) for i in range(Der.shape[0])]
            target_genes = [j_basis + "_" + str(i) for i in range(Der.shape[1])]
        else:
            source_genes, target_genes = adata_.var_names, adata_.var_names
    else:
        Der, source_genes, target_genes = intersect_sources_targets(
            regulators,
            regulators_,
            effectors,
            effectors_,
            Der if jacobian_gene is None else jacobian_gene,
        )

    df = pd.DataFrame(index=adata.obs_names[cell_indx])
    for i, source in enumerate(source_genes):
        for j, target in enumerate(target_genes):
            J = Der[j, i, :]  # dim 0: target; dim 1: source
            key = source + "->" + target + "_jacobian"
            df[key] = np.nan
            df.loc[:, key] = J

    return df

In [ ]:
Krumsiek_11_net = pd.read_csv(
    'https://raw.githubusercontent.com/theislab/scanpy/master/scanpy/sim_models/krumsiek11.txt', sep=r'\s+',
    skiprows=np.arange(24))
Krumsiek_11_net = Krumsiek_11_net.replace(
    {
        "Pu.1": "Spi1",
         "Fog1": "ZFPM1",
         "EKLF": "KLF1",
         "SCL": "TAL1",
         "cJun": "Jun",
         "EgrNab": "NAB2"
                     })

adata_labeling_11_genes.layers['velocity_S'] = adata_labeling[:, adata_labeling_11_genes.var_names].layers['velocity_alpha_minus_gamma_s'].copy()

for ind, i in Krumsiek_11_net.iterrows():
    i = i[1].upper(), i[0].upper() # 0 is target while 1 is regulator 
    if i[0] != i[1]:
        print(dyn.pl.jacobian(adata_labeling, x=i[0], y=i[1], layer='M_t',
                regulators=i[0],
                effectors=i[1]))
        print(dyn.pl.streamline_plot(adata_labeling_11_genes, color='cell_type', x=i[0].upper(), y=i[1].upper(), layer='M_t', ekey='M_s', 
                               vkey='velocity_alpha_minus_gamma_s'))

        J_df = get_jacobian(adata_labeling,
                             i[0],
                             i[1],
                             )
        
        jkey = i[0] + '->' + i[1] + '_jacobian'
        adata_labeling.obs['jacobian'] = J_df[jkey]

        print(dyn.pl.scatters(adata_labeling, 
                        color='jacobian', 
                        cmap='bwr', 
                        sym_c=True, 
                        basis=r"$E_{%s}\ vs.\ \frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[1], i[0]),
                        x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                        y=J_df[jkey].values.flatten()))
        
        print(dyn.pl.scatters(adata_labeling, 
                color='jacobian', 
                cmap='bwr', 
                basis=r"$E_{%s}\ vs.\ f_{%s}}$" % (i[0], i[1]),
                sym_c=True, 
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=adata_labeling[:, 'GATA1'].layers['velocity_alpha_minus_gamma_s'].A.flatten()))

In [ ]:
Krumsiek_11_net

In [ ]:
i = ["SPI1", "GATA1"]

dyn.pl.streamline_plot(adata_labeling_11_genes, 
                                       color='cell_type',
                                       x=i[0].upper(), 
                                       y=i[1].upper(), 
                                       layer='M_t', 
                                       ekey='M_s', 
                                       vkey='velocity_alpha_minus_gamma_s',
                      )

fig1, f1_axes = plt.subplots(ncols=6, nrows=2, constrained_layout=True, figsize=(24, 8)) # 
# 1. velocity

f1_axes[0, 0] = dyn.pl.scatters(adata_labeling, 
                                color=i[0],
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f^{alpha}_{%s}}$" % (i[0]),
                                sym_c=True, 
                                frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                ax=f1_axes[0, 0], save_show_or_return='return',
                               )
f1_axes[1, 0] = dyn.pl.scatters(adata_labeling, 
                                color=i[1],
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f^{alpha}_{%s}}$" % (i[1]),
                                sym_c=True, 
                                frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['M_t'].A.flatten(),
                                ax=f1_axes[1, 0], 
                                save_show_or_return='return',)

# 2. jacobian

J_df = get_jacobian(adata_labeling,
                     i[0],
                     i[0],
                     )

jkey_i0_i0_ = i[0] + '->' + i[0] + '_jacobian'
jkey_i0_i0 = i[0] + '_' + i[0] + '_jacobian'

adata_labeling.obs[jkey_i0_i0] = J_df[jkey_i0_i0_]

f1_axes[0, 1] = dyn.pl.scatters(adata_labeling, vmin=0, vmax=100, 
                                color=jkey_i0_i0, 
                                cmap='bwr', 
                                sym_c=True, frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[0]),
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                ax=f1_axes[0, 1], 
                                save_show_or_return='return',
                               )

J_df = get_jacobian(adata_labeling,
                     i[1],
                     i[0],
                     )

jkey_i1_i0 = i[1] + '_' + i[0] + '_jacobian'
jkey_i1_i0_ = i[1] + '->' + i[0] + '_jacobian'
adata_labeling.obs[jkey_i1_i0] = J_df[jkey_i1_i0_]

f1_axes[1, 1] = dyn.pl.scatters(adata_labeling,  vmin=0, vmax=100, 
                                color=jkey_i1_i0, 
                                cmap='bwr', 
                                sym_c=True, frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[1]),
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                ax=f1_axes[1, 1], 
                                save_show_or_return='return',
                               )


# Expression vs. velocity 
f1_axes[0, 2] = dyn.pl.scatters(adata_labeling, 
                                color=i[0], vmin=0, vmax=100, 
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f_{%s}}$" % (i[0]),
                                sym_c=True, 
                                frontier=True, sort='abs', 
                                alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[0]].layers['velocity_alpha_minus_gamma_s'].A.flatten(),
                                ax=f1_axes[0, 2], 
                                save_show_or_return='return',)
        
f1_axes[1, 2] = dyn.pl.scatters(adata_labeling, 
                                color=i[0],  vmin=0, vmax=100, 
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f_{%s}}$" % (i[0]),
                                sym_c=True, 
                                frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[0]].layers['velocity_alpha_minus_gamma_s'].A.flatten(),
                                ax=f1_axes[1, 2], 
                                save_show_or_return='return',)
f1_axes[0, 3] = dyn.pl.scatters(adata_labeling, 
                                color=i[1],  vmin=0, vmax=100, 
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f_{%s}}$" % (i[1]),
                                sym_c=True, 
                                frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['velocity_alpha_minus_gamma_s'].A.flatten(),
                                ax=f1_axes[0, 3], 
                                save_show_or_return='return',)
        
f1_axes[1, 3] = dyn.pl.scatters(adata_labeling, 
                                color=i[1],  vmin=0, vmax=100, 
                                layer='velocity_alpha_minus_gamma_s', 
                                use_smoothed=False,
                                cmap='bwr', 
                                basis=r"$f_{%s}}$" % (i[1]),
                                sym_c=True, 
                                frontier=True, sort='abs',alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                y=adata_labeling[:, i[1]].layers['velocity_alpha_minus_gamma_s'].A.flatten(),
                                ax=f1_axes[1, 3], 
                                save_show_or_return='return',)

# Expression vs. jacobian 
f1_axes[0, 4] = dyn.pl.scatters(adata_labeling, 
                                color=jkey_i0_i0,   vmin=0, vmax=100, 
                                cmap='bwr', 
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[0]),
                                sym_c=True, frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling.obs[jkey_i0_i0].values,
                                ax=f1_axes[0, 4], 
                                save_show_or_return='return',
                               )



f1_axes[1, 4] = dyn.pl.scatters(adata_labeling, 
                                color=jkey_i1_i0,  vmin=0, vmax=100, 
                                cmap='bwr', 
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[1]),
                                sym_c=True, frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                                y=adata_labeling.obs[jkey_i1_i0].values,
                                ax=f1_axes[1, 4], 
                                save_show_or_return='return',
                               )

f1_axes[0, 5] = dyn.pl.scatters(adata_labeling, 
                                color=jkey_i0_i0,   vmin=0, vmax=100, 
                                cmap='bwr', 
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[0]),
                                sym_c=True, frontier=True, sort='abs', alpha=0.2, pointsize=0.1,
                                x=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                y=adata_labeling.obs[jkey_i0_i0].values,
                                ax=f1_axes[0, 5], 
                                save_show_or_return='return',
                               )
f1_axes[1, 5] = dyn.pl.scatters(adata_labeling,  vmin=0, vmax=100, 
                                color=jkey_i1_i0, 
                                cmap='bwr', 
                                basis=r"$\frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[1]),
                                sym_c=True, frontier=True, sort='abs', alpha=0.1, pointsize=0.1,
                                x=adata_labeling[:, i[1]].layers['M_t'].A.flatten(), 
                                y=adata_labeling.obs[jkey_i1_i0].values,
                                ax=f1_axes[1, 5], 
                                save_show_or_return='return',
                               )

plt.tight_layout()
plt.show()

In [ ]:
# del adata_labeling.obs[i[0]]
# del adata_labeling.obs[i[1]]

for ind, i in Krumsiek_11_net.loc[[6, 19], :].iterrows():
    i = i[1].upper(), i[0].upper() # 0 is target while 1 is regulator 
    if i[0] != i[1]:
        print(dyn.pl.jacobian(adata_labeling, x=i[0], y=i[1], layer='M_t',
                regulators=i[0],
                effectors=i[1]))
        print(dyn.pl.streamline_plot(adata_labeling_11_genes, color='cell_type', x=i[0].upper(), y=i[1].upper(), layer='M_t', ekey='M_s', 
                               vkey='velocity_alpha_minus_gamma_s'))

#         adata_labeling.obs[i[0]] = adata_labeling[:, i[0]].layers['velocity_alpha_minus_gamma_s'].A.flatten()
#         adata_labeling.obs[i[1]] = adata_labeling[:, i[1]].layers['velocity_alpha_minus_gamma_s'].A.flatten()
        print(dyn.pl.scatters(adata_labeling, 
                color=i[0],
                layer='velocity_alpha_minus_gamma_s', 
                use_smoothed=False,
                cmap='bwr', 
                basis=r"$f_{%s}}$" % (i[0]),
                sym_c=True, 
                frontier=True, sort='abs',
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=adata_labeling[:, 'GATA1'].layers['M_t'].A.flatten()))
        
        print(dyn.pl.scatters(adata_labeling, 
                color=i[1],
                layer='velocity_alpha_minus_gamma_s', 
                use_smoothed=False,
                cmap='bwr', 
                basis=r"$f_{%s}}$" % (i[1]),
                sym_c=True, 
                frontier=True, sort='abs',
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=adata_labeling[:, 'GATA1'].layers['M_t'].A.flatten()))
        
        J_df = get_jacobian(adata_labeling,
                             i[0],
                             i[1],
                             )
        
        jkey = i[0] + '->' + i[1] + '_jacobian'
        adata_labeling.obs['jacobian'] = J_df[jkey]

        print(dyn.pl.scatters(adata_labeling, 
                        color='jacobian', 
                        cmap='bwr', 
                        sym_c=True, 
                        basis=r"$E_{%s}\ vs.\ \frac{\partial f_{%s}}{\partial x_{%s}}$" % (i[0], i[1], i[0]),
                        x=adata_labeling[:, i[0]].layers['M_t'].A.flatten(), 
                        y=J_df[jkey].values.flatten()))
        
        print(dyn.pl.scatters(adata_labeling, 
                color='jacobian', 
                cmap='bwr', 
                basis=r"$E_{%s}\ vs.\ f_{%s}}$" % (i[0], i[1]),
                sym_c=True, 
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=adata_labeling[:, 'GATA1'].layers['velocity_alpha_minus_gamma_s'].A.flatten()))

In [ ]:
adata_labeling.obs['SPI1->GATA1_jacobian'] = J_df['SPI1->GATA1_jacobian']

dyn.pl.scatters(adata_labeling, 
                color='SPI1->GATA1_jacobian', 
                cmap='bwr', 
                sym_c=True, 
                basis=r"$E_{%s}\ vs.\ \frac{\partial f_{%s}}{\partial x_{%s}}$" % (regulators, effectors, regulators),
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=J_df['SPI1->GATA1_jacobian'].values.flatten())

In [ ]:
dyn.pl.scatters(adata_labeling, 
                contour=True,
                color='SPI1->GATA1_jacobian', 
                cmap='bwr', 
                basis=r"$E_{%s}\ vs.\ f_{%s}}$" % (regulators, effectors),
                sym_c=True, 
                x=adata_labeling[:, 'SPI1'].layers['M_t'].A.flatten(), 
                y=adata_labeling[:, 'GATA1'].layers['velocity_alpha_minus_gamma_s'].A.flatten())

In [ ]:
J_df['SPI1->GATA1_jacobian'].values.flatten()

# test the idea of separately alpha and gamma_s

In [ ]:
def alpha_minus_gamma_s(new, gamma, t, M_s):
    alpha = new.A.T / (1 - np.exp(- gamma.values[:, None] * t.values[None, :])) * gamma.values[:, None]
    gamma_s = gamma.values[:, None] * M_s.A.T
    alpha_minus_gamma_s = alpha - gamma_s

    return alpha, gamma_s, alpha_minus_gamma_s

alpha_3, gamma_s_3, time_3_tmp = alpha_minus_gamma_s(new[time_3_cells, :], time_3_gamma, t[time_3_cells], M_s[time_3_cells, :])
alpha_5, gamma_s_5, time_5_tmp = alpha_minus_gamma_s(new[time_5_cells, :], time_5_gamma, t[time_5_cells], M_s[time_5_cells, :])

alpha_tmp = adata_labeling.layers['velocity_N'].copy()

alpha = alpha_tmp[:, adata_labeling.var.use_for_pca].copy()
alpha[time_3_cells, :] = alpha_3.T
alpha[time_5_cells, :] = alpha_5.T
alpha_tmp[:, adata_labeling.var.use_for_pca] = alpha.copy()

adata_labeling.layers['alpha'] = alpha_tmp.copy()

gamma_tmp = adata_labeling.layers['velocity_N'].copy()

gamma_s = gamma_tmp[:, adata_labeling.var.use_for_pca].copy()
gamma_s[time_3_cells, :] = gamma_s_3.T
gamma_s[time_5_cells, :] = gamma_s_5.T
gamma_tmp[:, adata_labeling.var.use_for_pca] = gamma_s.copy()

adata_labeling.layers['gamma_s'] = gamma_tmp.copy()

adata_labeling.write_h5ad('./adata_labeling_for_jacobian_with_only_alpha.h5ad')

In [ ]:
adata_labeling[:, adata_labeling.var.use_for_pca].layers['alpha'][:5, :5].A, adata_labeling[:, adata_labeling.var.use_for_pca].layers['gamma_s'][:5, :5].A

In [ ]:
alpha_3.shape, gamma_s_3.shape, time_3_tmp.shape

In [ ]:
adata_labeling

In [ ]:
valid_cell_type = ['HSC', 'MEP-like', 'Meg', 'Ery', 'Bas']

df = adata_labeling.obs.query("cell_type in @valid_cell_type").loc[:, ['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'cell_type']]
df['cell_type'] = list(df['cell_type'])

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

# Plot the distribution of clarity ratings, conditional on carat
sns.displot(
    data=df,
    x="umap_ori_ddhodge_potential", hue="cell_type",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    palette="glasbey_white",
)

In [ ]:
valid_cell_type = ['Meg', 'Ery', 'Bas']

df = adata_labeling.obs.query("cell_type in @valid_cell_type").loc[:, ['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'cell_type']]
df['cell_type'] = list(df['cell_type'])

sns.displot(
    data=df,
    x="umap_ori_ddhodge_potential", hue="cell_type",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    palette="glasbey_white",
)
plt.xlim(0.5, 1.5)

In [ ]:
dyn.vf.speed(adata_labeling, basis='pca')
dyn.vf.speed(adata_labeling, basis='umap_ori')

adata_labeling

In [ ]:
#valid_cell_type = ['Meg', 'Ery', 'Bas']

df = adata_labeling.obs.loc[:, ['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'speed_pca', 'speed_umap_ori', 'cell_type']]
df['cell_type'] = list(df['cell_type'])

dyn.pl.scatters(adata_labeling, x=df['umap_ori_ddhodge_potential'].values, y=df['speed_umap_ori'].values, color='cell_type')
# plt.xlim(0.5, 1.5)
# plt.ylim(0, 0.15)

In [ ]:
# dyn.vf.VectorField(adata_labeling, )
valid_cell_type = ['Meg', 'Ery', 'Bas']

df = adata_labeling.obs.query("cell_type in @valid_cell_type").loc[:, ['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'cell_type']]
df['cell_type'] = list(df['cell_type'])

ax = sns.displot(
    data=df,
    x="umap_ori_ddhodge_potential", hue="cell_type",
    kind="kde",
    palette="glasbey_white",
)
plt.xlim(0.5, 1.5)
plt.ylim(0, 0.15)
# sns.displot(data=penguins, x="flipper_length_mm", hue="species", col="sex", kind="kde")


In [ ]:
valid_cell_type = ['Meg', 'Ery', 'Bas']

df = adata_labeling#.obs.query("cell_type in @valid_cell_type").loc[:, ['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'cell_type']]
df['cell_type'] = list(df['cell_type'])

ax = sns.regplot(x="umap_ori_ddhodge_potential", y="big_tip", data=tips,
                 logistic=True, n_boot=500, y_jitter=.03)

plt.xlim(0.5, 1.5)
plt.ylim(0, 0.15)

In [ ]:
dyn.pl.streamline_plot(adata_labeling, color=['pca_ddhodge_potential', 'umap_ori_ddhodge_potential', 'cell_type'], basis='umap_ori')

In [ ]:
dyn.pd.state_graph(adata_labeling, group='cell_type', basis='umap_ori') # the markov method 

In [ ]:
dyn.pl.state_graph(adata_labeling, group='cell_type', color='cell_type', basis='umap_ori', show_legend='on data')

In [ ]:
dyn.tl.neighbors(adata_labeling, basis='umap_ori', result_prefix='umap_ori_knn', n_neighbors=80)


In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from patsy import dmatrix

group = 'cell_type'
data = adata_labeling.obs
groups = adata_labeling.obs['cell_type']
uniq_cts, data[group] = groups.unique(), list(groups)

transition_matrix = adata_labeling.obsp['umap_ori_knn_distances']
cell_membership = csr_matrix(dmatrix(f"~{group}+0", data=data))
membership_matrix = cell_membership.T.dot(transition_matrix).dot(cell_membership)
group_graph = adata_labeling.uns['cell_type_graph']['group_graph']

tmp = ['Bas', 'Ery', 'GMP-like', 'HSC', 'MEP-like', 'Meg', 'Mon', 'Neu']
membership_df = pd.DataFrame(membership_matrix.A> 0, index=tmp, columns=tmp)
group_graph_df = pd.DataFrame(group_graph, index=uniq_cts, columns=uniq_cts)
adata_labeling.uns['cell_type2_graph'] = adata_labeling.uns['cell_type_graph'].copy()
adata_labeling.uns['cell_type2_graph']['group_graph'] = (group_graph  * (membership_df.loc[uniq_cts, uniq_cts].values > 0) > 0).astype(float) # (group_graph  * (membership_matrix > 0) > 0).astype(float)

adata_labeling.obs['cell_type2'] = adata_labeling.obs['cell_type'].copy() 
dyn.pl.state_graph(adata_labeling, group='cell_type2', keep_only_one_direction=False, transition_threshold=None, color='cell_type2', basis='umap_ori', show_legend='on data')

In [ ]:
data[group], np.sort(tmp)

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from patsy import dmatrix

group = 'cell_type'
data = adata_labeling.obs

transition_matrix = adata_labeling.obsp['umap_ori_knn_distances']
cell_membership = csr_matrix(dmatrix(f"~{group}+0", data=data))
membership_matrix = cell_membership.T.dot(transition_matrix).dot(cell_membership)
group_graph = adata_labeling.uns['cell_type_graph']['group_graph']

membership_df = pd.DataFrame(membership_matrix.A> 0, 
             index=['Bas', 'Ery', 'GMP-like', 'HSC', 'MEP-like', 'Meg', 'Mon', 'Neu'], 
             columns=['Bas', 'Ery', 'GMP-like', 'HSC', 'MEP-like', 'Meg', 'Mon', 'Neu'])
uniq_cts = adata_labeling.obs['cell_type'].unique()
group_graph_df = pd.DataFrame(group_graph, 
             index=uniq_cts, 
             columns=uniq_cts)
adata_labeling.uns['cell_type2_graph'] = adata_labeling.uns['cell_type_graph'].copy()
adata_labeling.uns['cell_type2_graph']['group_graph'] = (group_graph  * (membership_df.loc[uniq_cts, uniq_cts].values > 0) > 0).astype(float)

adata_labeling.obs['cell_type2'] = adata_labeling.obs['cell_type'].copy() 
dyn.pl.state_graph(adata_labeling, group='cell_type2', keep_only_one_direction=False, transition_threshold=None, color='cell_type2', basis='umap_ori', show_legend='on data')

In [ ]:
uniq_cts

In [ ]:
source = list(uniq_cts).index('HSC')
target = [list(uniq_cts).index(i) for i in ['Mon', 'Meg', 'Ery', 'Bas', 'Neu']]

In [ ]:
M

In [ ]:
from scipy.sparse.csgraph import shortest_path

M = (group_graph  * (membership_df.loc[uniq_cts, uniq_cts].values > 0) > 0).astype(float)
M = adata_labeling.uns['cell_type2_graph']['group_graph']
M[M > 0] = 1 - M[M > 0]

D, Pr = shortest_path(M, directed=False, method='FW', return_predecessors=True)
res = np.zeros(M.shape)

def get_path(Pr, i, j):
    path = [j]
    k = j
    while Pr[i, k] != -9999:
        path.append(Pr[i, k])
        k = Pr[i, k]
    return path[::-1]

for j in target:
    p = j
    while Pr[source, p] != -9999:
        res[Pr[source, p], p] = 1
        p = Pr[source, p]
    
adata_labeling.uns['cell_type2_graph']['group_graph'] = res

dyn.pl.state_graph(adata_labeling, group='cell_type2', keep_only_one_direction=False, transition_threshold=None, color='cell_type2', basis='umap_ori', show_legend='on data')

In [ ]:
np.zeros_like(M)

for i, j in enumerate(Pr[source]):
    if j != -9999:
        p = Pr[source, j]
    if p != -9999:
        res[j, p] = 1
        
res, Pr

In [ ]:
membership_df.loc["GMP-like", 'MEP-like'] = 0
membership_df.loc["MEP-like", 'Mon'] = 0
membership_df.loc["Mon", 'Neu'] = 0
membership_df.loc["Mon", 'Bas'] = 0
membership_df.loc["Ery", 'Bas'] = 0
membership_df.loc["Ery", 'Meg'] = 0

adata_labeling.uns['cell_type2_graph']['group_graph'] = (group_graph  * (membership_df.loc[uniq_cts, uniq_cts].values > 0) > 0).astype(float)

adata_labeling.obs['cell_type2'] = adata_labeling.obs['cell_type'].copy() 
dyn.pl.state_graph(adata_labeling, group='cell_type2', keep_only_one_direction=False, transition_threshold=None, color='cell_type2', basis='umap_ori', show_legend='on data')

In [ ]:
(group_graph  * (membership_df.loc[uniq_cts, uniq_cts].values > 0) > 0).astype(float)

In [ ]:
#group_graph * 
pd.DataFrame(membership_matrix.A> 0, 
             index=['Bas', 'Ery', 'GMP-like', 'HSC', 'MEP-like', 'Meg', 'Mon', 'Neu'], 
             columns=['Bas', 'Ery', 'GMP-like', 'HSC', 'MEP-like', 'Meg', 'Mon', 'Neu'])

In [ ]:
cell_membership.A[:5, :], adata_labeling.obs.cell_type[:5]


In [ ]:
dyn.pd.state_graph(adata_labeling, group='cell_type', method='markov', transition_mat_key='cosine_transition_matrix') # the markov method 

In [ ]:
dyn.pl.state_graph(adata_labeling, group='cell_type', color='cell_type', basis='umap_ori')

In [ ]:
adata_labeling

In [ ]:
dyn.tl.cell_velocities(adata_labeling,
                       X=adata_labeling.layers['M_t'],
                       V=adata_labeling.layers['velocity_alpha_minus_gamma_s'],
                       basis='umap',
                       method='fp',
                       enforce=True,
                       )

adata_labeling

In [ ]:
dyn.pd.state_graph(adata_labeling,
                   group='cell_type',
                   method='markov',
                   transition_mat_key='fp_transition_matrix') # the markov method
dyn.pl.state_graph(adata_labeling, group='cell_type', basis='umap_ori', color='cell_type', show_legend='on data')


In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree
X = adata_labeling.uns['cell_type_graph']['group_graph']
Tcsr = minimum_spanning_tree(X)

adata_labeling.uns['cell_type_graph']['group_graph'] = Tcsr.A
dyn.pl.state_graph(adata_labeling, group='cell_type', basis='umap_ori', color='cell_type', show_legend='on data')


In [ ]:
Tcsr